In [ ]:
from scipy.stats import kstest
import pandas as pd
import numpy as np
import seaborn as sns
import itertools

sns.set_theme(style="ticks", palette="pastel")
sns.set(font='serif')
%matplotlib inline

### Can the indicators differentiate between metaheurstics with different behaviour?

This involves a statistical test (Kolmogorov-Smirnov) to see if two sets of indicator results are statistically similar. The "good" result is to see dissimilar results for metaheuristics that have both different topologies and different update functions, and less dissimilar results for metaheuristics that differ in one aspect.

In [1]:
tags = ["experiment","dimension","population_size","total_iterations","function","metaheuristic","control_parameters","run"]
diversity = ["DRoC_A","DRoC_B","ERT_Diversity","Critical_Diversity"]
fitness = ["FRoC_A","FRoC_B","ERT_Fitness","Critical_Fitness"]
separation = ["SRoC_A","SRoC_B","ERT_Separation","Critical_Separation"]
mobility = ["MRoC_A","MRoC_B","ERT_Mobility","Critical_Mobility"]
STN = ["ntotal","nbest","nshared","best_strength"]
IN = ["MID","MGC","SNID"]
others = ["EXPLORE","INFEASIBLE"]

indicators = diversity + fitness + separation + mobility + STN + IN + others
columns = tags + indicators
data = pd.read_csv("../../Results/characteristicTest/results.csv", names=columns)

In [2]:
metaheuristics = list(np.unique(data['metaheuristic']))
metaheuristics

['GBestPSO',
 'GreedyPSO',
 'Random20PSO',
 'RandomPSO',
 'RingPSO',
 'VonNeumannPSO']

In [3]:
paired_metaheuristics = list(itertools.combinations(metaheuristics, 2))
metaheuristics_short = {
    'GBestPSO': "sPSO",
    'Random20PSO': "20PSO",
    'RingPSO': "rPSO",
    'VonNeumannPSO': "vnPSO",
    'GreedyPSO': "gPSO",
    'RandomPSO': 'iPSO'
}
neighbourhood_difference = [
    'sPSO - 20PSO',
    'sPSO - rPSO',
    'sPSO - vnPSO',
    '20PSO - rPSO',
    '20PSO - vnPSO',
    'rPSO - vnPSO'
]
update_difference = [
    'sPSO - gPSO',
    'sPSO - iPSO',
    'gPSO - iPSO'
]
double_difference = [
    '20PSO - iPSO',
    'iPSO - rPSO',
    'iPSO - vnPSO',
    'gPSO - 20PSO',
    'gPSO - vnPSO',
    'gPSO - rPSO'
]
paired_metaheuristics_string = [metaheuristics_short[pair[0]] + " - " + metaheuristics_short[pair[1]] for pair in paired_metaheuristics]
paired_metaheuristics_string

['sPSO - gPSO',
 'sPSO - 20PSO',
 'sPSO - iPSO',
 'sPSO - rPSO',
 'sPSO - vnPSO',
 'gPSO - 20PSO',
 'gPSO - iPSO',
 'gPSO - rPSO',
 'gPSO - vnPSO',
 '20PSO - iPSO',
 '20PSO - rPSO',
 '20PSO - vnPSO',
 'iPSO - rPSO',
 'iPSO - vnPSO',
 'rPSO - vnPSO']

In [4]:
functions = list(np.unique(data['function']))
functions

['AttractiveSector',
 'Brown',
 'Discus',
 'Ellipsoidal',
 'Elliptic',
 'Exponential',
 'GeneralizedDropWave',
 'GeneralizedEggCrate',
 'GeneralizedPrice2',
 'Mishra1',
 'NeedleEye',
 'Pinter',
 'Qing',
 'Rosenbrock',
 'SchwafferN7',
 'Schwefel_1.2',
 'Step3Function',
 'Weierstrass']

In [21]:
function_pretty = {
    'AttractiveSector': "Attractive Sector",
    'Brown': "Brown",
    'Discus': "Discus",
    'Ellipsoidal': "Ellipsoidal",
    'Elliptic': "Elliptic",
    'Exponential': "Exponential",
    'GeneralizedDropWave': "Drop Wave",
    'GeneralizedEggCrate': "Egg Crate",
    'GeneralizedPrice2': "Price 2",
    'Mishra1': "Mishra 1",
    'NeedleEye': "Needle Eye",
    'Pinter': "Pinter",
    'Qing': "Qing",
    'Rosenbrock': "Rosenbrock",
    'SchwafferN7': "Schwaffer 7",
    'Schwefel_1.2': "Schwefel $1.2$",
    'Step3Function': "Step 3",
    'Weierstrass': "Weierstrass",
}
indicator_pretty = {
    "DRoC_A": "DRoC Type A",
    "DRoC_B": "DRoC Type B",
    "ERT_Diversity": "ERT Diversity",
    "Critical_Diversity": "Critical Diversity",
    "FRoC_A": "FRoC Type A",
    "FRoC_B": "FRoC Type B",
    "ERT_Fitness": "ERT Fitness",
    "Critical_Fitness": "Critical Fitness",
    "SRoC_A": "SRoC Type A",
    "SRoC_B": "SRoC Type B",
    "ERT_Separation": "ERT Separation",
    "Critical_Separation": "Critical Separation",
    "MRoC_A": "MRoC Type A",
    "MRoC_B": "MRoC Type B",
    "ERT_Mobility": "ERT Mobility",
    "Critical_Mobility": "Critical Mobility",
    "ntotal": "ntotal",
    "nbest": "nbest",
    "nshared": "nshared",
    "best_strength": "best-strength",
    "MID": "Mean ID",
    "MGC": "Mean GC",
    "SNID": "SNID",
    "EXPLORE": "EXPLORE%",
    "INFEASIBLE": "INFEASIBLE%"
}

# Preliminary investigation

Before performing statistical tests, we want an idea of what our data looks like. We want an idea of whether Nan's are something we should be worried abou, and how much variation there is.

In [6]:
df = data[["function", "metaheuristic"] + indicators]
nan_sum = df.isna().sum()
nan_sum[nan_sum > 0]

MRoC_A    1
MRoC_B    1
dtype: int64

In [7]:
# given the low number in comparison to the size of the data, we will just drop these rows
df = df.dropna(inplace=False)

In [8]:
distinct_values = df.groupby(["function", "metaheuristic"]).nunique()
distinct_values = distinct_values.reset_index(inplace=False)

If we split the results into unique sets of function and metaheuristic, how often is there only a single distinct value for an indicator in that set?

In [9]:
percentage_non_distinct_all = pd.DataFrame(columns=['Indicator', 'Percentage'])
for indicator in indicators:
    value_count = distinct_values[indicator].value_counts()
    if 1 in value_count:
        percent = value_count[1] / (len(distinct_values)) * 100
    else:
        percent = 0
    percentage_non_distinct_all.loc[len(percentage_non_distinct_all.index)] = [indicator, percent]
percentage_non_distinct_all

,Indicator,Percentage
0,DRoC_A,0.000000
1,DRoC_B,0.000000
2,ERT_Diversity,0.000000
3,Critical_Diversity,0.000000
4,FRoC_A,0.000000
5,FRoC_B,0.000000
6,ERT_Fitness,0.000000
7,Critical_Fitness,0.000000
8,SRoC_A,0.000000
9,SRoC_B,0.000000


What does this mean?  Well, we know that there is very little variation within the 30 runs for a unique choice of function and metaheuristic for nbest and best_strength. We don't yet know if there is also little variation between choices of function and metaheuristic. Little variation for different runs of the same experiment is not necessarily a bad thing, it, but little variation between different experiments is bad. To test this second point, we will use a statistical test.

# Statistical Test

### Kolmogorov–Smirnov

We will perform our statistical test for every pair of metaheuristics, for every indicator, for every function. We aggregate our results by counting R's (reject the null hypothesis) and F's (fail to reject the null hypothesis).

In [10]:
paired_columns = sum([[f"{pair} stat", f"{pair} p-val"] for pair in paired_metaheuristics_string], [])
full_results_df = pd.DataFrame(columns=["indicator", "function"] + paired_columns)
result_df = pd.DataFrame(columns=["indicator", "function"] + paired_metaheuristics_string)
for indicator in indicators:
    for function in functions:
        function_df = df[df["function"] == function]
        results = [indicator, function]
        full_results = [indicator, function]
        for alg1, alg2 in paired_metaheuristics:
            result = kstest(
                rvs=function_df[function_df['metaheuristic'] == alg1][indicator],
                cdf=function_df[function_df['metaheuristic'] == alg2][indicator],
            )
            full_results.append(result[0])
            full_results.append(result[1])
            if float(result[1]) < 0.05:
                results.append("R")
            else:
                results.append("F")
        result_df.loc[len(result_df.index)] = results
        full_results_df.loc[len(full_results_df.index)] = full_results
result_df["function"] = result_df["function"].apply(lambda x: function_pretty[x])
full_results_df["function"] = full_results_df["function"].apply(lambda x: function_pretty[x])
result_df.set_index(keys=["indicator", "function"], inplace=True)
full_results_df.set_index(keys=["indicator", "function"], inplace=True)
full_results_df.head(6)

/home/lauren/git/code_masters/behaviouralBenchmarking/.venv2/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:394: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  return hypotest_fun_in(*args, **kwds)
/home/lauren/git/code_masters/behaviouralBenchmarking/.venv2/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:394: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  return hypotest_fun_in(*args, **kwds)
/home/lauren/git/code_masters/behaviouralBenchmarking/.venv2/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:394: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  return hypotest_fun_in(*args, **kwds)
/home/lauren/git/code_masters/behaviouralBenchmarking/.venv2/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:394: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  return hypotest_fun_in(*args, **kw

sPSO - gPSO stat  sPSO - gPSO p-val  \
indicator function                                                 
DRoC_A    Attractive Sector          0.633333           0.000006   
          Brown                      0.566667           0.000087   
          Discus                     0.400000           0.015643   
          Ellipsoidal                0.333333           0.070888   
          Elliptic                   0.500000           0.000900   
          Exponential                0.533333           0.000293   

                             sPSO - 20PSO stat  sPSO - 20PSO p-val  \
indicator function                                                   
DRoC_A    Attractive Sector           0.200000            0.594071   
          Brown                       0.200000            0.594071   
          Discus                      0.666667            0.000001   
          Ellipsoidal                 0.200000            0.594071   
          Elliptic                    0.300000            0.135004   
          Exponential                 0.166667            0.807963   

                             sPSO - iPSO stat  sPSO - iPSO p-val  \
indicator function                                                 
DRoC_A    Attractive Sector               1.0       1.691123e-17   
          Brown                           1.0       1.691123e-17   
          Discus                          1.0       1.691123e-17   
          Ellipsoidal                     1.0       1.691123e-17   
          Elliptic                        1.0       1.691123e-17   
          Exponential                     1.0       1.691123e-17   

                             sPSO - rPSO stat  sPSO - rPSO p-val  \
indicator function                                                 
DRoC_A    Attractive Sector          0.533333           0.000293   
          Brown                      0.433333           0.006548   
          Discus                     0.666667           0.000001   
          Ellipsoidal                0.600000           0.000024   
          Elliptic                   0.433333           0.006548   
          Exponential                0.333333           0.070888   

                             sPSO - vnPSO stat  sPSO - vnPSO p-val  ...  \
indicator function                                                  ...   
DRoC_A    Attractive Sector           0.300000            0.135004  ...   
          Brown                       0.233333            0.392945  ...   
          Discus                      0.633333            0.000006  ...   
          Ellipsoidal                 0.300000            0.135004  ...   
          Elliptic                    0.400000            0.015643  ...   
          Exponential                 0.233333            0.392945  ...   

                             20PSO - rPSO stat  20PSO - rPSO p-val  \
indicator function                                                   
DRoC_A    Attractive Sector           0.400000        1.564339e-02   
          Brown                       0.366667        3.458008e-02   
          Discus                      0.266667        2.390730e-01   
          Ellipsoidal                 0.800000        8.466416e-10   
          Elliptic                    0.600000        2.366488e-05   
          Exponential                 0.333333        7.088799e-02   

                             20PSO - vnPSO stat  20PSO - vnPSO p-val  \
indicator function                                                     
DRoC_A    Attractive Sector            0.200000             0.594071   
          Brown                        0.200000             0.594071   
          Discus                       0.166667             0.807963   
          Ellipsoidal                  0.333333             0.070888   
          Elliptic                     0.266667             0.239073   
          Exponential                  0.200000             0.594071   

                             iPSO - rPSO stat  iPSO - rPSO p-val  \
indicator function        

In [11]:
result_df.head(6)

sPSO - gPSO sPSO - 20PSO sPSO - iPSO sPSO - rPSO  \
indicator function                                                             
DRoC_A    Attractive Sector           R            F           R           R   
          Brown                       R            F           R           R   
          Discus                      R            R           R           R   
          Ellipsoidal                 F            F           R           R   
          Elliptic                    R            F           R           R   
          Exponential                 R            F           R           F   

                            sPSO - vnPSO gPSO - 20PSO gPSO - iPSO gPSO - rPSO  \
indicator function                                                              
DRoC_A    Attractive Sector            F            R           R           R   
          Brown                        F            R           R           R   
          Discus                       R            R           R           R   
          Ellipsoidal                  F            R           R           R   
          Elliptic                     R            R           R           R   
          Exponential                  F            R           R           R   

                            gPSO - vnPSO 20PSO - iPSO 20PSO - rPSO  \
indicator function                                                   
DRoC_A    Attractive Sector            R            R            R   
          Brown                        R            R            R   
          Discus                       F            R            F   
          Ellipsoidal                  R            R            R   
          Elliptic                     R            R            R   
          Exponential                  R            R            F   

                            20PSO - vnPSO iPSO - rPSO iPSO - vnPSO  \
indicator function                                                   
DRoC_A    Attractive Sector             F           R            R   
          Brown                         F           R            R   
          Discus                        F           R            R   
          Ellipsoidal                   F           R            R   
          Elliptic                      F           R            R   
          Exponential                   F           R            R   

                            rPSO - vnPSO  
indicator function                        
DRoC_A    Attractive Sector            F  
          Brown                        F  
          Discus                       F  
          Ellipsoidal                  R  
          Elliptic                     R  
          Exponential                  F

In [12]:
small_table = pd.DataFrame(columns=[""] + paired_metaheuristics_string)
for indicator in indicators:
    failure_rate = [indicator]
    for metaheuristic in paired_metaheuristics_string:
        counts = result_df.loc[indicator][metaheuristic].value_counts()
        if "F" in counts:
            failure_rate.append(round(counts["F"] / 18 * 100, 1))
        else:
            failure_rate.append(0)
    small_table.loc[len(small_table.index)] = failure_rate
small_table.set_index(keys=[""], inplace=True)
small_table_t = small_table.transpose()
small_table_t

,DRoC_A,DRoC_B,ERT_Diversity,Critical_Diversity,FRoC_A,FRoC_B,ERT_Fitness,Critical_Fitness,SRoC_A,SRoC_B,...,Critical_Mobility,ntotal,nbest,nshared,best_strength,MID,MGC,SNID,EXPLORE,INFEASIBLE
sPSO - gPSO,33.3,22.2,22.2,27.8,33.3,55.6,44.4,50.0,44.4,72.2,...,33.3,50.0,50.0,27.8,27.8,5.6,5.6,0.0,22.2,22.2
sPSO - 20PSO,88.9,33.3,16.7,16.7,38.9,38.9,55.6,27.8,33.3,11.1,...,16.7,0.0,50.0,0.0,27.8,11.1,16.7,22.2,0.0,5.6
sPSO - iPSO,0.0,33.3,0.0,0.0,38.9,0.0,38.9,0.0,11.1,5.6,...,0.0,0.0,27.8,0.0,27.8,27.8,27.8,11.1,0.0,0.0
sPSO - rPSO,27.8,11.1,0.0,16.7,22.2,0.0,33.3,0.0,16.7,5.6,...,27.8,0.0,55.6,11.1,22.2,0.0,0.0,0.0,0.0,0.0
sPSO - vnPSO,61.1,33.3,0.0,22.2,38.9,16.7,44.4,11.1,33.3,5.6,...,16.7,0.0,50.0,5.6,27.8,0.0,0.0,16.7,0.0,0.0
gPSO - 20PSO,16.7,11.1,0.0,0.0,55.6,11.1,44.4,11.1,11.1,22.2,...,22.2,5.6,33.3,44.4,22.2,5.6,5.6,0.0,0.0,0.0
gPSO - iPSO,0.0,5.6,5.6,0.0,11.1,5.6,5.6,0.0,0.0,5.6,...,0.0,0.0,55.6,0.0,55.6,5.6,5.6,5.6,0.0,0.0
gPSO - rPSO,0.0,5.6,0.0,5.6,33.3,0.0,27.8,5.6,0.0,11.1,...,5.6,0.0,33.3,5.6,22.2,0.0,0.0,0.0,0.0,0.0
gPSO - vnPSO,11.1,11.1,0.0,0.0,44.4,0.0,50.0,0.0,16.7,0.0,...,11.1,5.6,33.3,11.1,22.2,0.0,0.0,0.0,0.0,0.0
20PSO - iPSO,0.0,44.4,0.0,0.0,33.3,5.6,22.2,0.0,16.7,22.2,...,5.6,0.0,22.2,0.0,22.2,0.0,22.2,0.0,0.0,5.6


In [22]:
new_index = []
alt_coloured_table = small_table_t.astype(str)
alt_coloured_table.columns = [indicator_pretty[x] for x in alt_coloured_table.columns]
for i in alt_coloured_table.index:
    if i in neighbourhood_difference:
        alt_coloured_table.loc[i] = alt_coloured_table.loc[i].apply(lambda x: "\\textcolor{mymint}{" + f"{x}" + "}")
        new_index.append("\\textcolor{mymint}{" + f"{i}" + "}")
    elif i in update_difference:
        alt_coloured_table.loc[i] = alt_coloured_table.loc[i].apply(lambda x: "\\textcolor{myfuchsia}{" + f"{x}" + "}")
        new_index.append("\\textcolor{myfuchsia}{" + f"{i}" + "}")
    elif i in double_difference:
        alt_coloured_table.loc[i] = alt_coloured_table.loc[i].apply(lambda x: "\\textcolor{mybrown}{" + f"{x}" + "}")
        new_index.append("\\textcolor{mybrown}{" + f"{i}" + "}")
alt_coloured_table.index = new_index
alt_coloured_table = alt_coloured_table.transpose()
alt_coloured_table.columns = ["\\rotatebox{90}{" + c +"}" for c in alt_coloured_table.columns]
alt_coloured_table = alt_coloured_table.sort_index(axis=1)
with pd.option_context("max_colwidth", None):
    print(alt_coloured_table.to_latex(label=f"tab:RQDistinct", escape=False,  caption=f"ummarised results of two-sample Kolmogorov-Smirnov test for each pair of metaheuristics. The null hypothesis states that the two samples are drawn from the same distribution, the alternative hypothesis states that the two samples are not from the same distribution. The table tallies the percentage rate at which the test fails to reject the null hypothesis, when tested for 30 runs for each of the 18 benchmark functions. Values coloured in pink indicate metaheuristic pairs that differ by an update function, values in green differ by social topology, and values in brown differ in both update function and social topology."))

\begin{table}
\caption{ummarised results of two-sample Kolmogorov-Smirnov test for each pair of metaheuristics. The null hypothesis states that the two samples are drawn from the same distribution, the alternative hypothesis states that the two samples are not from the same distribution. The table tallies the percentage rate at which the test fails to reject the null hypothesis, when tested for 30 runs for each of the 18 benchmark functions. Values coloured in pink indicate metaheuristic pairs that differ by an update function, values in green differ by social topology, and values in brown differ in both update function and social topology.}
\label{tab:RQDistinct}
\begin{tabular}{llllllllllllllll}
\toprule
 & \rotatebox{90}{\textcolor{mybrown}{20PSO - iPSO}} & \rotatebox{90}{\textcolor{mybrown}{gPSO - 20PSO}} & \rotatebox{90}{\textcolor{mybrown}{gPSO - rPSO}} & \rotatebox{90}{\textcolor{mybrown}{gPSO - vnPSO}} & \rotatebox{90}{\textcolor{mybrown}{iPSO - rPSO}} & \rotatebox{90}{\textcol